In [46]:
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
import pgrtk
pgrtk.__version__

# Load index
sdb = pgrtk.SeqIndexDB()
sdb.load_from_agc_index("/data/data/pgr-tk-HGRP-y1-evaluation-set-v0")

In [77]:
def load_sub_seq(loci:tuple, compliment=False, database=sdb) -> list:
    """ Reverse complement a sequence as a list of bytes (unsigned 8bit interger).

    Parameters
    ----------
    database    : loaded in .mdb object
    loci        : tuple containing ('genome_file', 'chromosome', start, end)
    compliment  : bool indicating whether the sequence should be complimented or not
    Returns
    -------
    list of bytes 
        the list of bytes of the given subsequence

    """
    subseq = database.get_sub_seq(loci[0], loci[1], loci[2], loci[3])
    if compliment: subseq = pgrtk.rc_byte_seq(subseq)
    return subseq

def get_seq_list(ROI_seq: list, sdb=sdb, padding: int=1.5e5, plot=True) -> list:
    '''
    Get a list of sequences from sdb index that match the ROI_seq.
    :param sdb: SeqIndexDB object, the PGR index
    :param ROI_seq: ROI sequence
    :param padding: padding length  
    :param plot:  
    '''

    ROI_len = len(ROI_seq)

    # query the PGR index to find matches of this ROI
    query_results = pgrtk.query_sdb(sdb, ROI_seq)
    seq_list = []
    seq_info = sdb.seq_info.copy() # Cache seq_info to reduce Rust HashMap to Python dictionary conversion in a loop
    i = 1
    if plot:
        plt.figure(figsize=(36, 36))
    for k in list(query_results.keys()):
        ctg_name, source, _ = seq_info[k]
        seq_id = k
        rgns = query_results[k].copy()
        # rgns = pgrtk.merge_regions(rgns,tol=1000) # if additional region merge needed
        for rgn in rgns:
            b, e, length, orientation, aln = rgn #beginning, end, length, orientation, alignments
            aln.sort()
            # Filters out reads that are not the entire length of the gene.
            #if aln[0][0][0] > padding or aln[-1][0][1] < padding + ROI_len: 
            #    continue
            
            seq =  sdb.get_sub_seq(source, ctg_name, b, e)
            # print(source, ctg_name, b, e)
            if orientation == 1:
                seq = pgrtk.rc_byte_seq(seq)

            seq_list.append(("{}_{}_{}_{}".format(ctg_name, b, e, orientation), seq))
            
            if e-b < len(ROI_seq) * 0.50: # ignore partial match
                continue
            else:
                x, y = pgrtk.get_shmmr_dots(ROI_seq, seq, 32, 32, 1, 1)
                if i <= 25 and plot:
                    plt.subplot(5,5,i)
                    plt.plot(x, y, ".", markersize=0.1)
                    ylabel = "#".join(ctg_name.split("#")[:2])
                    if len(ylabel) > 20:
                        ylabel = ylabel[:20]
                    plt.ylabel(ylabel)
                    # plt.plot([padding, padding],[0, max(y)], "r")
                    # plt.plot([padding+ROI_len, padding+ROI_len],[0, max(y)], "r")
                i += 1

    return seq_list

In [78]:
# Assign Subsequences
# (Genome File Name, Chromosome Name, Seq Start Location, Seq End Location)
C4 = ('chm13_tagged.fa', "chr6_chm13", 31835263, 31855887)
GATM = ('chm13_tagged.fa', "chr15_chm13", 43169299, 43186634)
HLA = ('chm13_tagged.fa', 'chr6_chm13', 32016331, 32813515)
CFTR = ('chm13_tagged.fa', 'chr7_chm13', 118831112, 118894842)

# Get seqs
C4_seq = load_sub_seq(C4)
HLA_seq = load_sub_seq(HLA)
GATM_seq = load_sub_seq(GATM)
CFTR_seq = load_sub_seq(CFTR)

# Get seq list
C4_list = get_seq_list(C4_seq, plot=False)
GATM_list = get_seq_list(GATM_seq, plot=False)
HLA_list = get_seq_list(HLA_seq, plot=False)
CFTR_list = get_seq_list(CFTR_seq, plot=False)

In [39]:
!mkdir -p data/region_graphs
!mkdir -p figures

In [80]:
ROI_lists = [C4_list, GATM_list, HLA_list, CFTR_list]
seq_names = ["C4", "GATM", "HLA", "CFTR"]

for seq_name, ROI_list in zip(seq_names, ROI_lists):
    new_sdb = pgrtk.SeqIndexDB()
    
    new_sdb.load_from_seq_list(ROI_list, w=80, k=56, r=2, min_span=18)
    '''
    w: window size, 1 minimizer per window, increase to analyze larger structures more 
        efficiently. 
    k: minimizer size, (should stay smaller than window size)
    r: hierarchical reduction factor, 
    min_span: Minimum distance between minimizers.
    '''
    new_sdb.generate_mapg_gfa(0, f"data/region_graphs/{seq_name}.gfa")

# Graph entropy metrics

In [83]:
for seq_name, ROI_list in zip(seq_names, ROI_lists):
    try:
        (entropy, weight_list) = pgrtk.compute_graph_diffusion_entropy(f"data/region_graphs/{seq_name}.gfa", max_nodes = 6000)
        print("%s %s"%(seq_name, entropy))
    except: print("failed on %s"%seq_name)

C4 7.882602
GATM 7.498429
failed on HLA
CFTR 9.326647


# Plotting subgraphs

In [81]:
%%bash
input="data/region_graphs/"
output="figures/"
for graph in $(ls data/region_graphs) ; do 
    image=${graph/".gfa"/".png"}
    echo "$output""$image"
    Bandage image "$input""$graph" "$output""$image"  --edgewidth 1.0 --edgecol blue --colour random --outline 0.5 
    done

figures/C4.png
figures/CFTR.png
figures/GATM.png
figures/HLA.png


C4
<img src="figures/C4.png">

GATM
<img src="figures/GATM.png">

CFTR
<img src="figures/CFTR.png">

HLA
<img src="figures/HLA.png">